In [4]:
import numpy as np
import fixed_income_derivatives_E2025 as fid
import matplotlib.pyplot as plt
from scipy.optimize import minimize

r0, a, b, sigma = 0.032, 1.2, 0.048, 0.1
T_max, T_mesh = 10, 0.1

# Problem 2 - ZCB prices, spot rates and instantaneous forward rates.
T = np.array([i*T_mesh for i in range(0,int(T_max/T_mesh)+1)])
p = fid.zcb_price_cir(r0,a,b,sigma,T)
R = fid.spot_rate_cir(r0,a,b,sigma,T)
f = fid.forward_rate_cir(r0,a,b,sigma,T)

# Fitting the CIR model (To itself)
param_0 = 0.03, 0.5, 0.04, 0.04
result = minimize(fid.fit_cir_obj,param_0,method = 'nelder-mead',args = (R,T),options={'xatol': 1e-20,'disp': True})
print(f"Problem 2. Parameters from the fit: {result.x}. SSE of the fit: {result.fun}")

sigma = 0.1
param_0 = 0.03, 0.5, 0.04
result = minimize(fid.fit_cir_no_sigma_obj,param_0,method = 'nelder-mead',args = (sigma,R,T),options={'xatol': 1e-20,'disp': True})
print(f"Problem 2. Parameters from the fit when sigma=0.1 is assumed: {result.x}. SSE of the fit: {result.fun}")


# Problem 3 - Swap rates for a swap paying 6M Euribor against an annual fixed rate
T_6m = np.array([i*0.5 for i in range(int(T_max/0.5)+1)])
p_6m = fid.zcb_price_cir(r0,a,b,sigma,T_6m)
f_6m = fid.forward_rates_from_zcb_prices(T_6m,p_6m,horizon = 1,method = "simple")
T_swap = np.array([i for i in range(0,T_max+1)])
R_swap = np.nan*np.ones([T_max+1])
for i in range(1,T_max+1):
    R_swap[i], S = fid.swap_rate_from_zcb_prices(0,0,T_swap[i],"annual",T,p,"semiannual",L = 0)
print(f"Problem 3a. 10Y par swap rate: {R_swap[10]}")
print(f"Problem 3b. All swap rates: {R_swap}")

# Problem 4 - Simulation of the short rate in the Vasicek model
seed = np.random.seed(13)
M_simul, T_simul = 1000, 10
size_ci = 0.95
r_simul = fid.simul_cir(r0,a,b,sigma,M_simul,T_simul,method = "exact",seed = seed)
t_simul = np.array([i*(T_simul/M_simul) for i in range(0,M_simul+1)])
lb, ub = fid.ci_cir(r0,a,b,sigma,t_simul,size_ci,type_ci = "two_sided")
mu_sd, sigma_sd = fid.mean_cir(r0,a,b,sigma,np.inf), fid.stdev_vasicek(r0,a,b,sigma,np.inf)
lb_sd = fid.ci_cir(r0,a,b,sigma,np.inf,size_ci,type_ci = "two_sided")[0]
ub_sd = fid.ci_cir(r0,a,b,sigma,np.inf,size_ci,type_ci = "two_sided")[1]
print(f"Problem 4. Simulated short rate with {size_ci} confidence bands and stationary distribution.")


# Problem 5
M_simul, T_simul, N_simul = 5000, 5, 1000
alpha = 1
strike = 0.045
T_caplet = np.array([i for i in range(2,T_simul+1)])
chi_caplet = np.zeros([4])
Mps = int(M_simul/T_simul)
for n in range(0,N_simul):
    r_simul_caplet = fid.simul_cir(r0,a,b,sigma,M_simul,T_simul,method = "exact",seed = None)
    sum_disc = sum(r_simul_caplet[0:int((T_caplet[0]-alpha)*Mps)])
    for i, T_i in enumerate(T_caplet):
        r_caplet = r_simul_caplet[int(T_i-alpha)*Mps]
        p_caplet = fid.zcb_price_cir(r_caplet,a,b,sigma,alpha)
        L_caplet = 1/p_caplet - 1
        # print(f"n: {n}, r_caplet: {r_caplet}, p_caplet: {p_caplet}, L_caplet: {L_caplet}")
        sum_disc += sum(r_simul_caplet[int((T_i-alpha)*Mps):int(T_i*Mps)])
        chi_caplet[i] += np.exp(sum_disc/Mps)*max(L_caplet - strike,0)/N_simul
print(f"Problem 5. Caplet prices: {chi_caplet*10000}")



/var/folders/ch/svk174ms2t7404n3sl5_kg380000gn/T/ipykernel_777/1426243612.py:17: RuntimeWarning: Maximum number of function evaluations has been exceeded.
  result = minimize(fid.fit_cir_obj,param_0,method = 'nelder-mead',args = (R,T),options={'xatol': 1e-20,'disp': True})
/var/folders/ch/svk174ms2t7404n3sl5_kg380000gn/T/ipykernel_777/1426243612.py:22: RuntimeWarning: Maximum number of function evaluations has been exceeded.
  result = minimize(fid.fit_cir_no_sigma_obj,param_0,method = 'nelder-mead',args = (sigma,R,T),options={'xatol': 1e-20,'disp': True})


Problem 2. Parameters from the fit: [0.03200169 1.20685591 0.04789409 0.06067821]. SSE of the fit: 1.1396449848451726e-11
Problem 2. Parameters from the fit when sigma=0.1 is assumed: [0.032 1.2   0.048]. SSE of the fit: 2.4256428018551283e-28
Problem 3a. 10Y par swap rate: 0.04735380703424437
Problem 3b. All swap rates: [       nan 0.0394124  0.04269522 0.04442669 0.04543051 0.04606412
 0.04649346 0.04680132 0.04703206 0.04721107 0.04735381]
caplet prices: [41.52784588 58.29909888 66.55403939 74.06389668]


In [5]:
# Fitting the term structure of ZCB spot rates.
EURIBOR_fixing = [{"id": 0,"instrument": "libor","maturity": 1/2, "rate":0.03723}]
fra_market = [{"id": 1,"instrument": "fra","exercise": 1/12,"maturity": 7/12, "rate": 0.03938},
{"id": 2,"instrument": "fra","exercise": 2/12,"maturity": 8/12, "rate": 0.04139},
{"id": 3,"instrument": "fra","exercise": 3/12,"maturity": 9/12, "rate": 0.04329},
{"id": 4,"instrument": "fra","exercise": 4/12,"maturity": 10/12, "rate": 0.04507},
{"id": 5,"instrument": "fra","exercise": 5/12,"maturity": 11/12, "rate": 0.04673},
{"id": 6,"instrument": "fra","exercise": 6/12,"maturity": 12/12, "rate": 0.04830},
{"id": 7,"instrument": "fra","exercise": 7/12,"maturity": 13/12, "rate": 0.04976},
{"id": 8,"instrument": "fra","exercise": 8/12,"maturity": 14/12, "rate": 0.05114},
{"id": 9,"instrument": "fra","exercise": 9/12,"maturity": 15/12, "rate": 0.05242}]
swap_market = [{"id": 10,"instrument": "swap","maturity": 2, "rate": 0.05089, "float_freq": "semiannual", "fixed_freq": "annual","indices": []},
{"id": 11,"instrument": "swap","maturity": 3, "rate": 0.05536, "float_freq": "semiannual", "fixed_freq": "annual","indices": []},
{"id": 12,"instrument": "swap","maturity": 4, "rate": 0.05786, "float_freq": "semiannual", "fixed_freq": "annual","indices": []},
{"id": 13,"instrument": "swap","maturity": 5, "rate": 0.05915, "float_freq": "semiannual", "fixed_freq": "annual","indices": []},
{"id": 14,"instrument": "swap","maturity": 7, "rate": 0.05985, "float_freq": "semiannual", "fixed_freq": "annual","indices": []},
{"id": 15,"instrument": "swap","maturity": 10, "rate": 0.05922, "float_freq": "semiannual", "fixed_freq": "annual","indices": []},
{"id": 16,"instrument": "swap","maturity": 15, "rate": 0.05776, "float_freq": "semiannual", "fixed_freq": "annual","indices": []},
{"id": 17,"instrument": "swap","maturity": 20, "rate": 0.05678, "float_freq": "semiannual", "fixed_freq": "annual","indices": []},
{"id": 18,"instrument": "swap","maturity": 30, "rate": 0.05579, "float_freq": "semiannual", "fixed_freq": "annual","indices": []}]
data = EURIBOR_fixing + fra_market + swap_market

# interpolation_options = {"method":"linear","transition": "smoo"}
interpolation_options = {"method":"hermite","degree":3,"transition": "smooth"}
# interpolation_options = {"method":"nelson_siegel","transition": "smooth"}

T_fit, R_fit = fid.zcb_curve_fit(data,interpolation_options = interpolation_options)
T_inter = np.array([i*1/12 for i in range(0,361)])
p_inter, R_inter, f_inter, T_inter = fid.zcb_curve_interpolate(T_inter,T_fit,R_fit,interpolation_options = interpolation_options)

# Fitting the CIR model to market data
param_0 = 0.03, 0.5, 0.04, 0.04
result = minimize(fid.fit_cir_obj,param_0,method = 'nelder-mead',args = (R_inter,T_inter),options={'xatol': 1e-20,'disp': True})
print(f"Parameters from the fit: {result.x}. SSE of the fit: {result.fun}")
r0_fit, a_fit, b_fit, sigma_fit = result.x
R_vas_fit = fid.spot_rate_cir(r0_fit, a_fit, b_fit, sigma_fit,T_inter)


# PLot of zcb prices, spot rates and forward rates in the CIR model
fig = plt.figure(constrained_layout=False,dpi=300,figsize=(5,3))
fig.suptitle(f"Spot- and forward rates in the CIR model",fontsize=10)
gs = fig.add_gridspec(nrows=1,ncols=1,left=0.12,bottom=0.2,right=0.88,top=0.90,wspace=0,hspace=0)
ax = fig.add_subplot(gs[0,0])
xticks = [0,2,4,6,8,10]
ax.set_xticks(xticks)
ax.set_xticklabels(xticks,fontsize = 6)
ax.set_xlim([xticks[0]-0.01,xticks[-1]+0.01])
plt.xlabel(f"Maturity",fontsize = 7)
yticks1 = [0,0.2,0.4,0.6,0.8,1]
ax.set_yticks(yticks1)
ax.set_yticklabels(yticks1,fontsize = 6)
ax.set_ylim([yticks1[0],yticks1[-1] + (yticks1[-1]-yticks1[0])*0.02])
plt.grid(axis = 'y', which='major', color=(0.7,0.7,0.7,0), linestyle='--')
ax.set_ylabel(f"ZCB Prices",fontsize = 7)
p1 = ax.scatter(T, p, s = 2, color = 'black', marker = ".",label="ZCB prices")
ax2 = ax.twinx()
yticks2 = [0,0.01,0.02,0.03,0.04,0.05]
ax2.set_yticks(yticks2)
ax2.set_yticklabels(yticks2,fontsize = 6)
ax2.set_ylim([yticks2[0],yticks2[-1]+(yticks2[-1]-yticks2[0])*0.02])
ax2.set_ylabel(f"Spot and forward rates",fontsize = 7)
p2 = ax2.scatter(T, R, s = 2, color = 'red', marker = ".",label="Spot rate")
p3 = ax2.scatter(T, f, s = 2, color = 'blue', marker = ".",label="Instantaneous forward rate")
plots = [p1,p2,p3]
labels = [item.get_label() for item in plots]
ax.legend(plots,labels,loc="lower right",fontsize = 5)
fig.savefig("/Users/PC/Documents/Dokumenter – Olivia's Macbook/9. Semester/Fixed Income/FixedIncomeCO/PLOTS/cir_zcb_plot.pdf")
plt.close(fig)
#plt.show()

# PLot of zcb prices, spot rates and forward rates in the CIR model
fig = plt.figure(constrained_layout=False,dpi=300,figsize=(5,3))
fig.suptitle(f"Swap- and 6M forward EURIBOR rates in the CIR model",fontsize=10)
gs = fig.add_gridspec(nrows=1,ncols=1,left=0.12,bottom=0.2,right=0.88,top=0.90,wspace=0,hspace=0)
ax = fig.add_subplot(gs[0,0])
xticks = [0,2,4,6,8,10]
ax.set_xticks(xticks)
ax.set_xticklabels(xticks,fontsize = 6)
ax.set_xlim([xticks[0]-0.01,xticks[-1]+0.01])
plt.xlabel(f"Maturity",fontsize = 7)
yticks = [0,0.01,0.02,0.03,0.04,0.05]
ax.set_yticks(yticks)
ax.set_yticklabels(yticks,fontsize = 6)
ax.set_ylim([yticks[0],yticks[-1]+(yticks[-1]-yticks[0])*0.02])
ax.set_yticks(yticks)
ax.set_yticklabels(yticks,fontsize = 6)
ax.set_ylim([yticks[0],yticks[-1] + (yticks[-1]-yticks[0])*0.02])
plt.grid(axis = 'y', which='major', color=(0.7,0.7,0.7,0), linestyle='--')
ax.set_ylabel(f"Swap rates and 6M forward EURIBOR rates",fontsize = 7)
p1 = ax.scatter(T_swap, R_swap, s = 2, color = 'black', marker = ".",label="Swap rates")
p2 = ax.scatter(T_6m, f_6m, s = 2, color = 'orange', marker = ".",label="6M forward rate")
plots = [p1,p2]
labels = [item.get_label() for item in plots]
ax.legend(plots,labels,loc="lower right",fontsize = 5)
fig.savefig("/Users/PC/Documents/Dokumenter – Olivia's Macbook/9. Semester/Fixed Income/FixedIncomeCO/PLOTS/cir_swap_plot.pdf")
plt.close(fig)
#plt.show()

# PLot of simulated short rates in the CIR model
fig = plt.figure(constrained_layout=False,dpi=300,figsize=(5,3))
fig.suptitle(f"Simulated spot rates in the CIR model",fontsize=10)
gs = fig.add_gridspec(nrows=1,ncols=1,left=0.12,bottom=0.2,right=0.88,top=0.90,wspace=0,hspace=0)
ax = fig.add_subplot(gs[0,0])
xticks = [0,2,4,6,8,10]
ax.set_xticks(xticks)
ax.set_xticklabels(xticks,fontsize = 6)
ax.set_xlim([xticks[0]-0.01,xticks[-1]+0.01])
plt.xlabel(f"Time",fontsize = 7)
yticks1 = [0,0.02,0.04,0.06,0.08,0.1]
ax.set_yticks(yticks1)
ax.set_yticklabels(yticks1,fontsize = 6)
ax.set_ylim([yticks1[0],yticks1[-1] + (yticks1[-1]-yticks1[0])*0.02])
plt.grid(axis = 'y', which='major', color=(0.7,0.7,0.7,0), linestyle='--')
ax.set_ylabel(f"Short rate",fontsize = 7)
p1 = ax.scatter(t_simul, r_simul, s = 1, color = 'black', marker = ".",label="Simulated short rate")
p2 = ax.scatter(t_simul, lb, s = 1, color = 'red', marker = ".",label=f"Lower {size_ci} CB")
p3 = ax.scatter(t_simul, ub, s = 1, color = 'red', marker = ".",label=f"Upper {size_ci} CB")
p4 = ax.scatter(
    t_simul,
    np.full(len(t_simul), lb_sd),
    s=1,
    color='blue',
    marker='.',
    label=f"Lower {size_ci} CB Stat. dist."
)
p5 = ax.scatter(
    t_simul,
    np.full(len(t_simul), ub_sd),
    s=1,
    color='blue',
    marker='.',
    label=f"Upper {size_ci} CB Stat. dist."
)
plots = [p1,p2,p3,p4,p5]
labels = [item.get_label() for item in plots]
ax.legend(plots,labels,loc="upper right",fontsize = 5)
fig.savefig("/Users/PC/Documents/Dokumenter – Olivia's Macbook/9. Semester/Fixed Income/FixedIncomeCO/PLOTS/cir_simul_plot.pdf")
plt.close(fig)
#plt.show()


fig = plt.figure(constrained_layout=False, dpi = 300, figsize = (5,3))
fig.suptitle(f"Calibrated zero coupon spot rates", fontsize = 9)
gs = fig.add_gridspec(nrows=1,ncols=1,left=0.12,bottom=0.2,right=0.88,top=0.90,wspace=0,hspace=0)
ax = fig.add_subplot(gs[0,0])
xticks = [0,1,2,3,4,5,7,10,15,20,30]
ax.set_xticks(xticks)
ax.set_xticklabels(xticks,fontsize = 6)
ax.set_xlim([xticks[0]+-0.2,xticks[-1]+0.2])
plt.xlabel(f"Maturity",fontsize = 6)
ax.set_yticks([0,0.02,0.04,0.06,0.08])
ax.set_yticklabels([0,0.02,0.04,0.06,0.08],fontsize = 6)
ax.set_ylim([0,0.0825])
plt.grid(axis = 'y', which='major', color=(0.7,0.7,0.7,0), linestyle='--')
p1 = ax.scatter(T_inter, R_inter, s = 1, color = 'blue', marker = ".",label="Interpolated spot rates")
p2 = ax.scatter(T_inter, f_inter, s = 1, color = 'red', marker = ".",label="Interpolated forward rates")
p3 = ax.scatter(T_fit, R_fit, s = 1, color = 'black', marker = ".",label="Calibrated spot rates")
p4 = ax.scatter(T_inter, R_vas_fit, s = 1, color = 'green', marker = ".",label="CIR fitted spot rates")
plots = [p1,p2,p3,p4]
labels = [item.get_label() for item in plots]
ax.legend(plots,labels,loc="lower right",fontsize = 6)
bbox = {"facecolor": (1,1,1,0.8),"edgecolor": (0.7,0.7,0.7,0.5),"boxstyle": "Round"}
if interpolation_options["method"] == "hermite":
    ax.text(0.32,0.0023,f" method: {interpolation_options['method']} \n degree: {interpolation_options['degree']} \n transition: {interpolation_options['transition']}", fontsize = 6,linespacing = 1.7, bbox = bbox)
else:
    ax.text(0.32,0.0023,f" method: {interpolation_options['method']} \n transition: {interpolation_options['transition']}", fontsize = 6,linespacing = 1.7, bbox = bbox)
fig.savefig("/Users/PC/Documents/Dokumenter – Olivia's Macbook/9. Semester/Fixed Income/FixedIncomeCO/PLOTS/zcb_market_fit_plot.pdf")
plt.close(fig)
#plt.show()


Parameters from the fit: [ 0.02573074  2.71955742  0.05761604 -0.60048048]. SSE of the fit: 0.002059355012176282


/var/folders/ch/svk174ms2t7404n3sl5_kg380000gn/T/ipykernel_777/2830939.py:33: RuntimeWarning: Maximum number of function evaluations has been exceeded.
  result = minimize(fid.fit_cir_obj,param_0,method = 'nelder-mead',args = (R_inter,T_inter),options={'xatol': 1e-20,'disp': True})
